In [26]:
import os
import sys
import pandas as pd
sys.path.append("../ixf2json/")
import ixf2json

import pyodbc
import json
import sqlalchemy as sa

In [2]:
from IPython.core.display import display, HTML, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

In [5]:
os.listdir("..")

['.gitignore.swp',
 'example_notebooks',
 'json_data',
 'README.md',
 '.gitignore',
 'ixf2json',
 '.ipynb_checkpoints',
 'DB2',
 'environment.yaml',
 '.git',
 'cp4d-mortgage-artifacts',
 'creds']

In [6]:
os.listdir("../json_data/")[:10]

['tab83.json',
 'tab101.json',
 'tab140.json',
 'tab95.json',
 'tab117.json',
 'tab4.json',
 'tab56.json',
 'tab40.json',
 'tab17.json',
 'tab60.json']

In [7]:
log = {
    "success": [],
    "file_name_errors": [], 
    "file_conversion_errors": [],
    "file_extension_errors": []
    }

In [24]:
def convert_ixf():
    for file in os.listdir("../DB2/data"):
        file_list = file.split(".")
        if len(file_list) > 2:
            print(file)
            log["file_name_errors"].append(file)
        else:
            filename = file_list[0]
            extension = file_list[-1]
        if extension == "ixf":
            print("extension: ", extension)
            try:
                ixf2json.convert(f'../DB2/data/{file}', f'../json_data/{filename}.json')
                print(file)
                log["success"].append(file)
            except:
                print("error converting file")
                log["file_conversion_errors"].append(file)
                pass
        else:
            print("filename incorrect; not an ixf file.")
            log["file_extension_errors"].append(file)
            
    with open("log.json", "w") as f:
        json.dump(log, f)

In [25]:
convert_ixf()

extension:  ixf
error converting file


NameError: name 'log' is not defined

In [9]:
with open('log.json', 'r') as f:
     log = json.load(f)

In [10]:
log.keys()

dict_keys(['success', 'file_name_errors', 'file_conversion_errors', 'file_extension_errors'])

In [11]:
len(log["success"])

153

In [12]:
os.listdir("../json_data/")

['tab83.json',
 'tab101.json',
 'tab140.json',
 'tab95.json',
 'tab117.json',
 'tab4.json',
 'tab56.json',
 'tab40.json',
 'tab17.json',
 'tab60.json',
 'tab37.json',
 'tab21.json',
 'tab76.json',
 'tab99.json',
 'tab8.json',
 'tab137.json',
 'tab121.json',
 'tab120.json',
 'tab136.json',
 'tab9.json',
 'tab77.json',
 'tab98.json',
 'tab20.json',
 'tab36.json',
 'tab61.json',
 'tab16.json',
 'tab41.json',
 'tab57.json',
 'tab5.json',
 'tab116.json',
 'tab94.json',
 'tab141.json',
 'tab100.json',
 'tab82.json',
 'tab66.json',
 'tab89.json',
 'tab31.json',
 'tab27.json',
 'tab70.json',
 'tab131.json',
 'tab127.json',
 'tab85.json',
 'tab107.json',
 'tab150.json',
 'tab146.json',
 'tab93.json',
 'tab111.json',
 'tab2.json',
 'tab50.json',
 'tab46.json',
 'tab11.json',
 'tab10.json',
 'tab47.json',
 'tab51.json',
 'tab3.json',
 'tab110.json',
 'tab92.json',
 'tab147.json',
 'tab151.json',
 'test.json',
 'tab106.json',
 'tab84.json',
 'tab126.json',
 'tab130.json',
 'tab71.json',
 'tab26.js

## Parsing list that maps filenames to table/schema names

In [17]:
ref_lst = []

schema_set = set()

with open("../DB2/data/db2move.lst", "r") as f:
    for line in f.readlines():
        lst = line.split("!")
        schema, table = lst[1].split(".")
        schema = schema.replace('"', '' ).strip()
        table = table.replace('"', '' ).strip()
        file = lst[2]
        filename = lst[2].split(".")[0]
        result = dict(
            schema=schema,
            table=table,
            file=file,
            filename=filename
        )
        schema_set.add(schema)
        ref_lst.append(result)

In [18]:
ref_df = pd.DataFrame(ref_lst)

In [16]:
ref_df.shape

(151, 4)

In [17]:
ref_df.head()

,schema,table,file,filename
0,GOSALESRT,ACTIVITY_STATUS_LOOKUP,tab1.ixf,tab1
1,GOSALES,BRANCH,tab2.ixf,tab2
2,GOSALES,CONVERSION_RATE,tab3.ixf,tab3
3,GOSALES,COUNTRY,tab4.ixf,tab4
4,GOSALES,CURRENCY_LOOKUP,tab5.ixf,tab5


In [15]:
json_tables = os.listdir("../json_data/")

In [19]:
json_tables.sort()

In [20]:
# json_tables

In [21]:
len(json_tables)

152

In [22]:
schema_set

{'GOSALES', 'GOSALESCT', 'GOSALESDW', 'GOSALESHR', 'GOSALESMR', 'GOSALESRT'}

In [23]:
ref_df

,schema,table,file,filename
0,GOSALESRT,ACTIVITY_STATUS_LOOKUP,tab1.ixf,tab1
1,GOSALES,BRANCH,tab2.ixf,tab2
2,GOSALES,CONVERSION_RATE,tab3.ixf,tab3
3,GOSALES,COUNTRY,tab4.ixf,tab4
4,GOSALES,CURRENCY_LOOKUP,tab5.ixf,tab5
...,...,...,...,...
146,GOSALES,TIME_QUARTER_LOOKUP,tab147.ixf,tab147
147,GOSALESHR,TRAINING,tab148.ixf,tab148
148,GOSALESHR,TRAINING_DETAILS,tab149.ixf,tab149
149,GOSALES,XGOREV,tab150.ixf,tab150


### make sql connection

In [3]:
with open("../creds/creds.json", "r") as f:
    all_creds = json.load(f)

In [4]:
creds = all_creds["mysql"]

In [5]:
locals().update(creds)  # Creating local variables from the JSON entries for this database

In [6]:
connxn_string = f"mysql+mysqldb://{username}:{password}@{host}:{port}/{db}"

In [7]:
engine = sa.create_engine(
    connxn_string, 
)

In [8]:
with engine.connect() as con:
#     con.execute("USE master;") 
#     con.execute("CREATE DATABASE GSDB;")
    con.execute("USE GSDB;")

In [9]:
# q = engine.execute('SHOW DATABASES')
# available_tables = q.fetchall()

In [10]:
schema_set

NameError: name 'schema_set' is not defined

In [29]:
# # Create Schemata
# with engine.connect() as con:
#     for schema in schema_set:  
#         con.execute(f"CREATE SCHEMA {schema};")

In [30]:
for schema in schema_set:
    sa.schema.CreateSchema(schema)

In [11]:
existing_databases = engine.execute("SHOW DATABASES;")
# Results are a list of single item tuples, so unpack each tuple
existing_databases = [d[0] for d in existing_databases]

In [12]:
existing_databases

['GOSALES',
 'GOSALESCT',
 'GOSALESDW',
 'GOSALESHR',
 'GOSALESMR',
 'GOSALESRT',
 'GSDB',
 'defaultdbname',
 'information_schema',
 'mortgage',
 'mysql',
 'performance_schema']

In [13]:
insp = sa.inspect(engine)
db_list = insp.get_schema_names()
print(db_list)

['GOSALES', 'GOSALESCT', 'GOSALESDW', 'GOSALESHR', 'GOSALESMR', 'GOSALESRT', 'GSDB', 'defaultdbname', 'information_schema', 'mortgage', 'mysql', 'performance_schema']


In [14]:
ref_df.head()

NameError: name 'ref_df' is not defined

## Read in JSONs into pandas, display metadata, log errors

In [35]:
log["json_read_errors"] = []
ref_df["col_count"] = pd.NA
ref_df["row_count"] = pd.NA

for json_table in json_tables:
    filename = json_table.split(".")[0]
#     display(Markdown(f"### {filename}"))
    try:
        df = pd.read_json(f"../json_data/{json_table}", lines=False)
        row_ct, col_ct = df.shape
#         display(Markdown(f"#### Size: {df_shape}"))
        ref_df.loc[ref_df["filename"] == filename, ["row_count"]] = row_ct
        ref_df.loc[ref_df["filename"] == filename, ["col_count"]] = col_ct
    except:
        display(Markdown(f"#### Error reading file {json_table}"))
        log["json_read_errors"].append(json_table)
        pass
    

#### Error reading file tab110.json

#### Error reading file tab12.json

#### Error reading file tab137.json

In [37]:
ref_df.sort_values("row_count", ascending=False).head(10)

,schema,table,file,filename,col_count,row_count
87,GOSALES,ORDER_DETAILS,tab88.ixf,tab88,9,446023
140,GOSALESDW,SLS_SALES_ORDER_DIM,tab141.ixf,tab141,4,446023
139,GOSALESDW,SLS_SALES_FACT,tab140.ixf,tab140,17,446023
128,GOSALES,SALES_TARGET,tab129.ixf,tab129,9,233625
141,GOSALESDW,SLS_SALES_TARG_FACT,tab142.ixf,tab142,8,233625
79,GOSALESDW,MRK_PRODUCT_SURVEY_FACT,tab80.ixf,tab80,6,165074
101,GOSALESMR,PRODUCT_SURVEY_RESULTS,tab102.ixf,tab102,6,165074
59,GOSALESDW,FIN_FINANCE_FACT,tab60.ixf,tab60,6,164132
97,GOSALES,PRODUCT_FORECAST,tab98.ixf,tab98,7,129096
19,GOSALESDW,DIST_PRODUCT_FORECAST_FACT,tab20.ixf,tab20,7,129096


In [44]:
log["json_read_errors"]

['tab110.json', 'tab12.json', 'tab137.json']

In [45]:
len(log["json_read_errors"])

3

In [20]:
bad_jsons = set(log["json_read_errors"])

NameError: name 'log' is not defined

In [21]:
bad_jsons = {'tab110.json', 'tab12.json', 'tab137.json'}

## Insert data into SQL DB

In [48]:
log.keys()

dict_keys(['success', 'file_name_errors', 'file_conversion_errors', 'file_extension_errors', 'json_read_errors', 'sql_insert_errors'])

In [49]:
log["file_name_errors"]

['tab110a.001.xml', 'tab9a.001.lob']

In [50]:
log["file_conversion_errors"]

['tab123.ixf', 'tab137.ixf', 'tab110a.001.xml', 'tab9a.001.lob']

In [51]:
log["file_extension_errors"]

['gsdb_constraints.sql',
 'EXPORT.out',
 'GOSALES_VERSION.txt',
 'db2move.lst',
 'gsdb_readme.txt']

In [52]:
log["sql_insert_errors"] = []

In [58]:
json_table = json_tables[3]

In [59]:
df = pd.read_json(f"../json_data/{json_table}")

In [60]:
df.head()

,PRODUCT_SIZE_CODE,PRODUCT_SIZE_EN,PRODUCT_SIZE_DE,PRODUCT_SIZE_FR,PRODUCT_SIZE_JA,PRODUCT_SIZE_CS,PRODUCT_SIZE_DA,PRODUCT_SIZE_EL,PRODUCT_SIZE_ES,PRODUCT_SIZE_FI,...,PRODUCT_SIZE_MS,PRODUCT_SIZE_NL,PRODUCT_SIZE_NO,PRODUCT_SIZE_PL,PRODUCT_SIZE_PT,PRODUCT_SIZE_RU,PRODUCT_SIZE_SC,PRODUCT_SIZE_SV,PRODUCT_SIZE_TC,PRODUCT_SIZE_TH
0,800,Unspecified,Unbestimmt,Non défini,指定なし,Neurčeno,Uspecificeret,Μη καθορισμένο,Sin especificar,Määrittämätön,...,Tidak ditentukan,Ongespecificeerd,Uspesifisert,Nieokreślone,Não especificado,Не задано,未指定,Ospecificerad,未指定,ไม่ระบุ
1,801,12 mm,12 mm,12 mm,12 mm,12 mm,12 mm,12 χλστ.,12 mm,12 mm,...,12 mm,12 mm,12 mm,12 mm,12 mm,12 мм,12 mm,12 mm,12 mm,12 มม.
2,802,100 ml,100 ml,100 ml,100 ml,100 ml,100 ml,100 χλστ/λίτ.,100 ml,100 ml,...,100 ml,100 ml,100 ml,100 ml,100 ml,100 мл,100 ml,100 ml,100 ml,100 มล.
3,803,250 ml,250 ml,250 ml,250 ml,250 ml,250 ml,250 χλστ/λίτ.,250 ml,250 ml,...,250 ml,250 ml,250 ml,250 ml,250 ml,250 мл,250 ml,250 ml,250 ml,250 มล.
4,804,330 ml,330 ml,330 ml,330 ml,330 ml,330 ml,330 χλστ/λίτ.,330 ml,330 ml,...,330 ml,330 ml,330 ml,330 ml,330 ml,330 мл,330 ml,330 ml,330 ml,330 มล.


In [53]:
for json_table in json_tables:
    # read in JSON into pandas, display metadata
    filename = json_table.split(".")[0]
    display(Markdown(f"### {filename}"))
    if json_table not in bad_jsons:
        df = pd.read_json(f"../json_data/{json_table}", lines=False)
    display(Markdown(f"#### Size: {df.shape}"))
#     display(Markdown(f"#### Columns:"))
#     for col in df.columns:
#         print(col)
    
    # Lookup table name and schema name from reference df
    ref_row = ref_df.query("filename == @filename")
    schema = ref_row["schema"].values[0]
    display(Markdown(f"#### Schema: {schema}"))
    table = ref_row["table"].values[0]
    display(Markdown(f"#### Table Name: {table}"))
    
    
    # Write df to sql
    try:
        df.to_sql(
            con=engine,
            name=table, 
            schema=schema,
            if_exists="replace",
            method="multi",
            index=False
                 )
    except:
        log["sql_insert_errors"].upate(json_table)
        pass
    
    print("*" * 20)
    

### tab1

#### Size: (2, 23)

#### Schema: GOSALESRT

#### Table Name: ACTIVITY_STATUS_LOOKUP

********************


...

#### Table Name: PRODUCT_LINE

********************


### test

#### Size: (55, 24)

IndexError: index 0 is out of bounds for axis 0 with size 0

## Check to make sure it worked

In [22]:
for json_table in json_tables:
    # read in JSON into pandas, display metadata
    filename = json_table.split(".")[0]
    display(Markdown(f"### {filename}"))
    # if json_table not in bad_jsons:
    # Lookup table name and schema name from reference df
    ref_row = ref_df.query("filename == @filename")
    schema = ref_row["schema"].values[0]
    display(Markdown(f"#### Schema: {schema}"))
    table = ref_row["table"].values[0]
    display(Markdown(f"#### Table Name: {table}"))
    if json_table not in bad_jsons:
        df = pd.read_sql_table(table, con=engine, schema=schema)
        print(df.shape)
        display(df.head())

### tab83

#### Schema: GOSALESDW

#### Table Name: MRK_PROMOTION_FACT

(11034, 14)


,index,ORGANIZATION_KEY,ORDER_DAY_KEY,RTL_COUNTRY_KEY,EMPLOYEE_KEY,RETAILER_KEY,PRODUCT_KEY,PROMOTION_KEY,SALES_ORDER_KEY,UNIT_COST,UNIT_PRICE,UNIT_SALE_PRICE,SALE_TOTAL,GROSS_PROFIT
0,0,11121,20040112,90010,4008,7166,30002,5502,177745,6.62,12.53,8.77,9076.95,2225.25
1,1,11121,20040112,90010,4008,7166,30003,5503,177751,15.93,23.80,21.42,16407.72,4205.34
2,2,11121,20040112,90011,4011,6838,30002,5502,177887,6.62,12.53,8.77,5814.51,1425.45
3,3,11121,20040112,90012,4013,6737,30003,5503,177848,15.93,23.80,21.42,10602.90,2717.55
4,4,11121,20040112,90013,4045,7266,30002,5502,178003,6.62,12.53,8.77,11050.20,2709.00


...

### tab14

#### Schema: GOSALESCT

#### Table Name: CUST_ORDER_HEADER

(39389, 14)


,index,CUST_ORDER_NUMBER,CUST_ORDER_DATE,CUST_CODE,CUST_CC_ID,CRDT_METHOD_CODE,ORDER_METHOD_CODE,CUST_ORDER_STATUS_CODE,CUST_TOTAL_QUANTITY,CUST_UNIQUE_ITEMS,CUST_SUBTOTAL,CUST_SHIP_COST,CUST_SALES_TAX,CUST_TOTAL
0,0,100000,2004-01-01-22.16.50.017000,131072,10000,29,5,3,1,1,71.23,9.29,5.23,85.75
1,1,100002,2004-01-01-12.35.54.017000,131073,10001,28,5,3,2,2,2570.79,10.57,438.83,3020.19
2,2,100006,2004-01-26-13.37.07.017000,131075,10002,29,5,3,1,1,35.34,6.96,8.25,50.55
3,3,100008,2004-01-13-10.06.24.017000,131076,10003,25,5,3,1,1,802.75,10.96,81.37,895.08
4,4,100010,2004-01-25-10.15.02.017000,131077,10004,29,5,3,1,1,375.64,7.94,61.37,444.95


In [61]:
log.keys()

dict_keys(['success', 'file_name_errors', 'file_conversion_errors', 'file_extension_errors', 'json_read_errors', 'sql_insert_errors'])

In [62]:
log["sql_insert_errors"]

[]